In [ ]:
# !pip install arcgis

In [ ]:
import os
import logging
import time
from PIL import Image
from arcgis.gis import GIS

# Configuración básica de logging
logging.basicConfig(
    filename='descarga_arcgis.log',
    filemode='a',
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO
)

# Conexión a ArcGIS Online o Enterprise
try:
    gis = GIS("https://arcgismaps.pantaleon.com/portal", "uvg_2024", "Inicio2024.")
    logging.info("Conexión a ArcGIS establecida exitosamente.")
except Exception as e:
    logging.error(f"Error al conectar a ArcGIS: {e}")
    raise SystemExit(f"Error al conectar a ArcGIS: {e}")

# Encuentra el elemento Survey123 por su ID
survey_id = "e9eff4bf0dff4d199ff5b889d8fd7980"  # Reemplaza con el ID real de tu Survey
survey = gis.content.get(survey_id)
if not survey:
    logging.error(f"No se encontró el Survey con ID: {survey_id}")
    raise SystemExit(f"No se encontró el Survey con ID: {survey_id}")
logging.info(f"Survey encontrado: {survey.title}")

# Identificar la capa correcta basada en un campo específico
nombre_campo_clave = 'clasificacion'  # Reemplaza con un campo específico de tu formulario

capas = survey.layers
capa_correcta = None

for capa in capas:
    nombres_campos = [campo['name'] for campo in capa.properties.fields]
    if nombre_campo_clave in nombres_campos:
        capa_correcta = capa
        break

if capa_correcta:
    feature_layer = capa_correcta
    logging.info(f"Usando la capa: {feature_layer.properties.name}")
else:
    logging.error(f"No se encontró una capa que contenga el campo '{nombre_campo_clave}'.")
    raise SystemExit(f"No se encontró una capa que contenga el campo '{nombre_campo_clave}'.")

# Función para verificar si un adjunto es una imagen
def is_image_attachment(attachment):
    return attachment['contentType'].lower() in [
        'image/png',
        'image/jpeg',
        'image/jpg',
        'image/gif',
        'image/bmp',
        'image/tiff'
    ]

# Función para redimensionar las imágenes
def resize_image(image_path, size=(128, 128)):
    try:
        with Image.open(image_path) as img:
            img = img.resize(size)
            img.save(image_path)
        logging.info(f"Imagen redimensionada: {image_path}")
    except Exception as e:
        logging.error(f"Error al redimensionar la imagen {image_path}: {e}")

# Directorio base para almacenar las imágenes descargadas
base_images_dir = "arcgis-survey-images"
os.makedirs(base_images_dir, exist_ok=True)
logging.info(f"Directorio base para imágenes: {base_images_dir}")

# Lista para almacenar errores
errores = []

# Función para generar un nombre de archivo único basado en object_id y attachment_id
def generar_nombre_unico(attachment_name, object_id, attachment_id):
    name, ext = os.path.splitext(attachment_name)
    # Sanitizar el nombre del archivo para evitar caracteres inválidos
    name = "".join([c for c in name if c.isalpha() or c.isdigit() or c in (' ', '_', '-')]).rstrip()
    # Puedes personalizar el formato del nombre según tus necesidades
    nombre_unico = f"{name}_OID{object_id}_ATT{attachment_id}{ext}"
    return nombre_unico

# Función para obtener un nombre de archivo único si ya existe
def get_unique_filename(file_path):
    if not os.path.exists(file_path):
        return file_path
    else:
        base, extension = os.path.splitext(file_path)
        i = 1
        new_file_path = f"{base}({i}){extension}"
        while os.path.exists(new_file_path):
            i += 1
            new_file_path = f"{base}({i}){extension}"
        return new_file_path

# Función para procesar y descargar una imagen
def procesar_descargar_imagen(feature, attachment):
    # Ajustar los nombres de los campos según tu data
    object_id = feature.attributes.get('objectid') or feature.attributes.get('OBJECTID') or feature.attributes.get('ObjectID')
    if object_id is None:
        logging.error("No se encontró el campo 'objectid' en los atributos de la feature.")
        errores.append((None, attachment['name'], "Campo 'objectid' no encontrado"))
        return
    classification = feature.attributes.get('clasificacion') or feature.attributes.get('Clasificacion') or 'SinClasificacion'

    # Crear un directorio para la clasificación si no existe
    class_dir = os.path.join(base_images_dir, str(classification))
    os.makedirs(class_dir, exist_ok=True)

    if is_image_attachment(attachment):
        # Generar un nombre único para cada imagen
        image_filename = generar_nombre_unico(attachment['name'], object_id, attachment['id'])
        image_path = os.path.join(class_dir, image_filename)

        try:
            # Descargar el adjunto a la carpeta con el nombre original
            feature_layer.attachments.download(
                oid=object_id,
                attachment_id=attachment['id'],
                save_path=class_dir
                # No se utiliza file_name aquí
            )
            logging.info(f"Descargada imagen a {class_dir}")
            
            # El archivo descargado se llama attachment['name']
            downloaded_path = os.path.join(class_dir, attachment['name'])
            
            # Verificar si el archivo descargado existe
            if not os.path.exists(downloaded_path):
                logging.error(f"El archivo descargado no existe: {downloaded_path}")
                errores.append((object_id, attachment['name'], "Archivo descargado no encontrado"))
                return
            
            # Verificar si el archivo único ya existe y obtener un nombre único
            if os.path.exists(image_path):
                original_image_path = image_path
                image_path = get_unique_filename(image_path)
                logging.warning(f"El archivo {original_image_path} ya existía. Se guardará como {os.path.basename(image_path)}")
            
            # Renombrar el archivo descargado a la ruta con el nombre único
            os.rename(downloaded_path, image_path)
            logging.info(f"Imagen renombrada a {image_path}")

            # Redimensionar la imagen
            resize_image(image_path)
        except Exception as e:
            logging.error(f"Error al descargar o procesar la imagen {attachment['name']} para el object_id {object_id}: {e}")
            errores.append((object_id, attachment['name'], str(e)))

# Función para obtener todas las features utilizando object IDs
def get_all_features(layer):
    all_features = []
    try:
        # Obtener el campo de Object ID
        object_id_field = layer.properties.objectIdField
        oid_info = layer.query(return_ids_only=True)
        object_ids = oid_info['objectIds']
        if not object_ids:
            logging.error("No se pudieron obtener los Object IDs.")
            return []
        total = len(object_ids)
        logging.info(f"Total de features disponibles: {total}")

        # Procesar los Object IDs en lotes
        batch_size = 1000  # Puedes ajustar este valor según tus necesidades
        for i in range(0, total, batch_size):
            batch_ids = object_ids[i:i + batch_size]
            where_clause = f"{object_id_field} IN ({', '.join(map(str, batch_ids))})"
            query_result = layer.query(where=where_clause, out_fields='*')
            all_features.extend(query_result.features)
            logging.info(f"Recuperados {len(all_features)} de {total} features...")
    except Exception as e:
        logging.error(f"Error al obtener las features: {e}")
        raise
    return all_features

# Descargar imágenes desde Survey123 y organizar en carpetas según la clasificación
try:
    all_features = get_all_features(feature_layer)
    total_features = len(all_features)
    logging.info(f"Total de features obtenidos: {total_features}")
except Exception as e:
    logging.error(f"Error al realizar la consulta a la capa: {e}")
    raise SystemExit(f"Error al realizar la consulta a la capa: {e}")

# Descargar todas las imágenes secuencialmente
for feature in all_features:
    object_id = feature.attributes.get('objectid') or feature.attributes.get('OBJECTID') or feature.attributes.get('ObjectID')
    if object_id is None:
        logging.error("No se encontró el campo 'objectid' en los atributos de la feature.")
        errores.append((None, None, "Campo 'objectid' no encontrado"))
        continue
    attachments = feature_layer.attachments.get_list(oid=object_id)
    if not attachments:
        logging.info(f"No se encontraron adjuntos para el object_id: {object_id}")
        continue
    for attachment in attachments:
        procesar_descargar_imagen(feature, attachment)
        # Implementar una pequeña pausa para evitar posibles límites de la API
        time.sleep(0.05)  # Pausa de 50 ms

logging.info("Descarga de imágenes completada.")

# Mostrar errores si los hay
if errores:
    logging.warning(f"Total de errores: {len(errores)}")
    for oid, nombre, error in errores:
        logging.warning(f"Object ID {oid}, Archivo {nombre}: {error}")
else:
    logging.info("No se encontraron errores durante la descarga.")

# Verificar la integridad de las imágenes descargadas
total_descargadas = 0
for root, dirs, files in os.walk(base_images_dir):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff')):
            total_descargadas += 1

logging.info(f"Total de imágenes descargadas: {total_descargadas}")
print(f"Imágenes descargadas y organizadas por clasificación en: {base_images_dir}")
print(f"Total de imágenes descargadas: {total_descargadas}")

# Mostrar errores en la consola
if errores:
    print(f"\nTotal de errores durante la descarga: {len(errores)}")
    for oid, nombre, error in errores:
        print(f"Object ID {oid}, Archivo {nombre}: {error}")
else:
    print("\nNo se encontraron errores durante la descarga.")


In [ ]:
# # Configuración básica de logging
# logging.basicConfig(
#     filename='descarga_arcgis.log',
#     filemode='a',
#     format='%(asctime)s - %(levelname)s - %(message)s',
#     level=logging.INFO
# )

In [ ]:
# # Conexión a ArcGIS Online o Enterprise
# try:
#     gis = GIS("https://arcgismaps.pantaleon.com/portal", "uvg_2024", "Inicio2024.")
#     logging.info("Conexión a ArcGIS establecida exitosamente.")
# except Exception as e:
#     logging.error(f"Error al conectar a ArcGIS: {e}")
#     raise SystemExit(f"Error al conectar a ArcGIS: {e}")

In [ ]:
# # Encuentra el elemento Survey123 por su ID
# survey_id = "e9eff4bf0dff4d199ff5b889d8fd7980"  # Reemplaza con el ID real de tu Survey
# survey = gis.content.get(survey_id)
# if not survey:
#     logging.error(f"No se encontró el Survey con ID: {survey_id}")
#     raise SystemExit(f"No se encontró el Survey con ID: {survey_id}")
# logging.info(f"Survey encontrado: {survey.title}")

# feature_layer = survey.layers[0]  # Capa que contiene las respuestas
# print(feature_layer)

In [ ]:
# # Función para verificar si un adjunto es una imagen
# def is_image_attachment(attachment):
#     return attachment['contentType'].lower() in [
#         'image/png',
#         'image/jpeg',
#         'image/jpg',
#         'image/gif',
#         'image/bmp',
#         'image/tiff'
#     ]

In [ ]:
# # Función para redimensionar las imágenes
# def resize_image(image_path, size=(128, 128)):
#     try:
#         with Image.open(image_path) as img:
#             img = img.resize(size)
#             img.save(image_path)
#         logging.info(f"Imagen redimensionada: {image_path}")
#     except Exception as e:
#         logging.error(f"Error al redimensionar la imagen {image_path}: {e}")


In [ ]:
# # Directorio base para almacenar las imágenes descargadas
# base_images_dir = "arcgis-survey-images"
# os.makedirs(base_images_dir, exist_ok=True)
# logging.info(f"Directorio base para imágenes: {base_images_dir}")

# # Diccionario para almacenar las imágenes duplicadas
# duplicated_images = {}

# # Lista para almacenar errores
# errores = []

In [ ]:
# # Función para generar un nuevo nombre de archivo para duplicados
# def generate_unique_filename(base_path):
#     name, ext = os.path.splitext(base_path)
#     counter = 1
#     new_path = base_path
#     while os.path.exists(new_path):
#         new_path = f"{name}({counter}){ext}"
#         counter += 1
#     return new_path

In [ ]:
# Función para procesar y descargar una imagen
def procesar_descargar_imagen(feature, attachment):
    object_id = feature.attributes.get('objectid')
    classification = feature.attributes.get('clasificacion', 'SinClasificacion')
    
    # Crear un directorio para la clasificación si no existe
    class_dir = os.path.join(base_images_dir, str(classification))
    os.makedirs(class_dir, exist_ok=True)
    
    if is_image_attachment(attachment):
        original_path = os.path.join(class_dir, attachment['name'])
        image_path = generate_unique_filename(original_path)
        
        # Registrar si es una imagen duplicada
        if image_path != original_path:
            if original_path not in duplicated_images:
                duplicated_images[original_path] = [image_path]
            else:
                duplicated_images[original_path].append(image_path)
        
        try:
            logging.info(f"Descargando imagen {os.path.basename(image_path)} para el object_id: {object_id}")
            feature_layer.attachments.download(
                oid=object_id,
                attachment_id=attachment['id'],
                save_path=os.path.dirname(image_path)
            )
            # Renombrar el archivo descargado si es necesario
            downloaded_path = os.path.join(os.path.dirname(image_path), attachment['name'])
            if downloaded_path != image_path:
                os.rename(downloaded_path, image_path)
                logging.info(f"Imagen renombrada a {image_path}")
            
            # Redimensionar la imagen
            resize_image(image_path)
        except Exception as e:
            logging.error(f"Error al descargar o procesar la imagen {attachment['name']} para el object_id {object_id}: {e}")
            errores.append((object_id, attachment['name'], str(e)))

In [ ]:
# Descargar imágenes desde Survey123 y organizar en carpetas según la clasificación
try:
    query_result = feature_layer.query(return_all=True)
    total_features = len(query_result.features)
    logging.info(f"Total de features obtenidos: {total_features}")
except Exception as e:
    logging.error(f"Error al realizar la consulta a la capa: {e}")
    raise SystemExit(f"Error al realizar la consulta a la capa: {e}")


In [ ]:
# Parámetros para descarga en paralelo
max_workers = 10  # Puedes ajustar este número según los recursos de tu sistema y límites de la API
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futuros = []
    for feature in query_result.features:
        object_id = feature.attributes.get('objectid')
        attachments = feature_layer.attachments.get_list(oid=object_id)
        if not attachments:
            logging.info(f"No se encontraron adjuntos para el object_id: {object_id}")
            continue
        for attachment in attachments:
            futuros.append(executor.submit(procesar_descargar_imagen, feature, attachment))
            # Implementar una pequeña pausa para evitar posibles límites de la API
            time.sleep(0.05)  # Pausa de 50 ms
    
    # Monitorear el progreso
    for futuro in as_completed(futuros):
        pass  # Aquí podrías implementar seguimiento si lo deseas

logging.info("Descarga de imágenes completada.")

In [ ]:
# # Identificar la capa correcta basada en un campo específico
# nombre_campo_clave = 'clasificacion'  # Reemplaza con un campo específico de tu formulario

# capas = survey.layers
# capa_correcta = None

# for capa in capas:
#     nombres_campos = [campo['name'] for campo in capa.properties.fields]
#     if nombre_campo_clave in nombres_campos:
#         capa_correcta = capa
#         break

# if capa_correcta:
#     feature_layer = capa_correcta
#     print(f"Usando la capa: {feature_layer.properties.name}")
# else:
#     logging.error(f"No se encontró una capa que contenga el campo '{nombre_campo_clave}'.")
#     raise SystemExit(f"No se encontró una capa que contenga el campo '{nombre_campo_clave}'.")

In [ ]:
# # Mostrar listado de imágenes duplicadas
# if duplicated_images:
#     logging.info("Listado de imágenes duplicadas:")
#     for original, duplicates in duplicated_images.items():
#         logging.info(f"Original: {original}")
#         for dup in duplicates:
#             logging.info(f"  Duplicado: {dup}")
# else:
#     logging.info("No se encontraron imágenes duplicadas.")

# # Mostrar errores si los hay
# if errores:
#     logging.warning(f"Total de errores: {len(errores)}")
#     for oid, nombre, error in errores:
#         logging.warning(f"Object ID {oid}, Archivo {nombre}: {error}")
# else:
#     logging.info("No se encontraron errores durante la descarga.")

# # Verificar la integridad de las imágenes descargadas
# total_descargadas = 0
# for root, dirs, files in os.walk(base_images_dir):
#     for file in files:
#         if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff')):
#             total_descargadas += 1

# logging.info(f"Total de imágenes descargadas: {total_descargadas}")
# print(f"Imágenes descargadas y organizadas por clasificación en: {base_images_dir}")
# print(f"Total de imágenes descargadas: {total_descargadas}")

# # Mostrar listado de imágenes duplicadas en la consola
# if duplicated_images:
#     print("\nListado de imágenes duplicadas:")
#     for original, duplicates in duplicated_images.items():
#         print(f"Original: {original}")
#         for dup in duplicates:
#             print(f"  Duplicado: {dup}")
# else:
#     print("\nNo se encontraron imágenes duplicadas.")

# # Mostrar errores en la consola
# if errores:
#     print(f"\nTotal de errores durante la descarga: {len(errores)}")
#     for oid, nombre, error in errores:
#         print(f"Object ID {oid}, Archivo {nombre}: {error}")
# else:
#     print("\nNo se encontraron errores durante la descarga.")

In [ ]:
# import os

# # Diccionario para almacenar las imágenes duplicadas
# duplicated_images = {}

# # Función para generar un nuevo nombre de archivo para duplicados
# def generate_unique_filename(base_path, filename):
#     name, ext = os.path.splitext(filename)
#     counter = 1
#     new_path = base_path
#     while os.path.exists(new_path):
#         new_path = os.path.join(os.path.dirname(base_path), f"{name}({counter}){ext}")
#         counter += 1
#     return new_path

# # Descargar imágenes desde Survey123 y organizar en carpetas según la clasificación
# for feature in feature_layer.query().features:
#     try:
#         object_id = feature.attributes['objectid']
#         classification = feature.attributes['clasificacion']  # Clasificación de la plaga

#         # Crear un directorio para la clasificación si no existe
#         class_dir = os.path.join(base_images_dir, classification)
#         if not os.path.exists(class_dir):
#             os.makedirs(class_dir)
        
#         # Descargar los adjuntos (imágenes) para el object_id
#         attachments = feature_layer.attachments.get_list(oid=object_id)
#         if not attachments:
#             print(f"No se encontraron adjuntos para el object_id: {object_id}")
        
#         for attachment in attachments:
#             if is_image_attachment(attachment):
#                 original_path = os.path.join(class_dir, attachment['name'])
                
#                 # Generar un nombre único si la imagen ya existe
#                 image_path = generate_unique_filename(original_path, attachment['name'])
                
#                 # Registrar si es una imagen duplicada
#                 if image_path != original_path:
#                     if original_path not in duplicated_images:
#                         duplicated_images[original_path] = [image_path]
#                     else:
#                         duplicated_images[original_path].append(image_path)
                
#                 print(f"Descargando imagen {os.path.basename(image_path)} para el object_id: {object_id}")
#                 feature_layer.attachments.download(oid=object_id, attachment_id=attachment['id'], save_path=os.path.dirname(image_path))
#                 os.rename(os.path.join(os.path.dirname(image_path), attachment['name']), image_path)
#                 resize_image(image_path)
        
#     except Exception as e:
#         print(f"Error al procesar el object_id {object_id}: {e}")

# print(f"Imágenes descargadas y organizadas por clasificación en: {base_images_dir}")

# # Mostrar listado de imágenes duplicadas
# if duplicated_images:
#     print("\nListado de imágenes duplicadas:")
#     for original, duplicates in duplicated_images.items():
#         print(f"Original: {original}")
#         for dup in duplicates:
#             print(f"  Duplicado: {dup}")
# else:
#     print("\nNo se encontraron imágenes duplicadas.")